In [13]:
import torch.nn.functional as F
import torch
import gc
import numpy as np
from torch import Tensor
from torch.utils.data import DataLoader, Dataset, IterableDataset
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path
import pandas as pd
import os

### Text allignments (just example at this moment)

In [14]:

def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

tokenizer = AutoTokenizer.from_pretrained('models/LaBSE-tuned/final')
model = AutoModel.from_pretrained('models/LaBSE-tuned/final')
model = model.to("cuda:1")
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(501153, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [15]:
# text piece was taken from: https://rus4all.ru/lez/20190528/11021/Sneg.html

lez_text = \
"""
ЧIехи баде дакIардин кIане пичинин патавай чилел вегьенвай месел алай. Адан виш йис хьанвай. ЧIехи яру цуьквер алай читдин чин алай яргъандикай хкатна аквазвайди адан гъвечIи хъхьанвай чин ва кьуранвай кьве гъил тир. Амма адаз килигай касдин виле сифтени-сифте акьазвайди дерин ва кьадар авачир биришар тир. Ам вичин биришра аруш хьанвай хьиз аквадай. Адан вилера экв амачир. ЧIехи бадедин кьилел чIулав ягълух, адан кIаникайни рехи шуткьу алай. Гададиз чидай хьи, чIехи бадедин кьилел чIар алач, ада кьве югъ идалай вилик вичин кьил гададин дидедив мукIратIдивди кьаз тунай, бадеди, лагьайтIа, яни гъвечIи бадеди чIехи бубадин уьлгуьчдалди аламукьай чIарар хтун хъувунай. Гададиз мадни чидай хьи, чIехи бадеди шуткьу кьилел чIарар аламачирди чуьнуьхун патал ваъ, вичин къедда авай дишегьлидал шуткьу тахьун айиб тирвиляй алукIзавай. Жив къвадалди ам са гьилле хъсанзавайди тир, мецел-ванцел чан аламай, махар ахъайдай. Гададиз адан вири махар, риваятар, негъилар, кьисаяр, бязи маниярни хуралай чидай, ятIани, гьар гъилера чIехи бадеди таниш за затI ахъайдайла, са цIийи лишан, цIийи везин, са цIийи гаф алава хъжедай. «Яраб идан рикIе икьван ксар, девирар, вакъиаяр, гафар гьикI гьакьзаватIа?» – фикирардай гадади. Садра ада бадедиз и суал ганай. «Ам Аллагьдин пай я, чан бала, адаз ганвай». Гада и гафарин гъавурда бегьемдиз акьуначир, амма чIехи баде амай инсанриз ухшар туширди, кьетIенди тирди гадади аннамишнай.
"""

ru_text = \
"""
Прабабушка лежала в постели под окном на полу, ближе к печи. Ей было сто лет. Из-под шерстяного одеяла с ситцевым покрывалом в крупных красных цветах выглядывали лишь ее уменьшившееся лицо и высохшие руки. Прежде всего в глаза бросались глубокие бесчисленные морщины. Можно было подумать, что она запуталась в собственных морщинах. В ее глазах не осталось света. Голова была повязана черным платком, а под ним было серое шутку*. Мальчик знал, что у нее на голове нет волос: два дня назад она попросила мать мальчика ножницами остричь ей волосы, бабушка же младшая опасной бритвой дедушки сбрила то, что еще оставалось от волос. Еще мальчик знал: прабабушка надевала шутку не потому, что стеснялась оголенной головы, а потому, что женщине, соблюдающей горские приличия, не пристало быть без шутку. До того, как выпал снег, она еще чувствовала себя неплохо, рассказывала сказки, голос у нее был еще живой. Мальчик помнил все ее сказки, легенды, были и небылицы, даже песни, но каждый раз, когда она вновь начинала рассказывать уже знакомое, он замечал что-то новое, какой-либо образ, черточку или незнакомое слово. Мальчик думал: «Как в ее памяти умещается столько людей, эпох, событий и слов?» Однажды он задал этот вопрос младшей бабушке. «Всевышний наделил ее этим даром, сынок». Мальчик не совсем понял ­услышанное, но он осознал, что прабабушка не похожа на других людей, что она особенная.
"""

In [16]:
# for multilingual-e5-large
# lez_sentences = [f"query: {s.strip()}" for s in lez_text.split(".")]
# ru_sentences = [f"query: {s.strip()}" for s in ru_text.split(".")]

# for LaBSE you shouldn't use prefix "query: "
lez_sentences = [s.strip() for s in lez_text.split(".")]
ru_sentences = [s.strip() for s in ru_text.split(".")]

len(lez_sentences), len(ru_sentences)

(16, 15)

In [17]:
# Tokenize the input texts
batch_dict = tokenizer(lez_sentences, max_length=512, padding=True, truncation=True, return_tensors='pt').to("cuda:1")

outputs = model(**batch_dict)
lez_embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
lez_embeddings = F.normalize(lez_embeddings, p=2, dim=1)


In [18]:
# Tokenize the input texts
batch_dict = tokenizer(ru_sentences, max_length=512, padding=True, truncation=True, return_tensors='pt').to("cuda:1")

outputs = model(**batch_dict)
ru_embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
ru_embeddings = F.normalize(ru_embeddings, p=2, dim=1)


In [19]:
cos_sim = cosine_similarity(ru_embeddings.detach().cpu().numpy(), lez_embeddings.detach().cpu().numpy())

In [20]:
from scipy.optimize import linear_sum_assignment

# Hungarian algorithm: Solves linear sum assignment problem
# https://en.wikipedia.org/wiki/Assignment_problem
# This algorithms guarantees that a bipartitle matching will be found, where:
# Each sentence from the first set will be matched to exactly one sentence from the second (1:1).  
# And vice versa.


row_ids, col_ids = linear_sum_assignment(1 - cos_sim)

for i, j in zip(row_ids, col_ids):
    print(f"Pair: ({i}, {j}), Cos Sim: {cos_sim[i, j]}")
    print(f"ru: {ru_sentences[i]}")
    print(f"lez: {lez_sentences[j]}")

Pair: (0, 0), Cos Sim: 0.7824299931526184
ru: Прабабушка лежала в постели под окном на полу, ближе к печи
lez: ЧIехи баде дакIардин кIане пичинин патавай чилел вегьенвай месел алай
Pair: (1, 1), Cos Sim: 0.9530683755874634
ru: Ей было сто лет
lez: Адан виш йис хьанвай
Pair: (2, 2), Cos Sim: 0.7018488645553589
ru: Из-под шерстяного одеяла с ситцевым покрывалом в крупных красных цветах выглядывали лишь ее уменьшившееся лицо и высохшие руки
lez: ЧIехи яру цуьквер алай читдин чин алай яргъандикай хкатна аквазвайди адан гъвечIи хъхьанвай чин ва кьуранвай кьве гъил тир
Pair: (3, 3), Cos Sim: 0.6578536033630371
ru: Прежде всего в глаза бросались глубокие бесчисленные морщины
lez: Амма адаз килигай касдин виле сифтени-сифте акьазвайди дерин ва кьадар авачир биришар тир
Pair: (4, 4), Cos Sim: 0.7087996006011963
ru: Можно было подумать, что она запуталась в собственных морщинах
lez: Ам вичин биришра аруш хьанвай хьиз аквадай
Pair: (5, 5), Cos Sim: 0.9358047246932983
ru: В ее глазах не осталось с